# INE

In [1]:
#Leer CSV
def leer_archivo_rango_periodo(ruta_archivo, periodo):
    
    #leer archivo CSV
    ine_original = pd.read_csv(ruta_archivo, encoding='latin1',sep="\t")
    
    #crear copia del DF original
    ine_copia= ine_original.copy()
    
    #FIltrar el DataFrame para incluir solo las filas que contienen el periodo deseado
    ine_filtrado= ine_copia[ine_copia['Periodo'].str.contains(periodo)]
    
    ine_filtrado = ine_filtrado.reset_index(drop=True)
    
    return ine_filtrado

In [10]:
#Limpiar columnas

def procesar_columnas(df):

    column_rename_dict = {
        'ï»¿Totales Territoriales': 'Totales_Territoriales',
        'Comunidades y Ciudades AutÃ³nomas': 'Comunidades_Ciudades_Autonomas',
        'Provincias': 'Provincia',
        'Viajeros y pernoctaciones': 'Viajeros_Pernoctaciones',
        'Residencia: Nivel 1': 'Residencia_Nivel_1',
        'Residencia: Nivel 2': 'Residencia_Nivel_2',
        'Periodo': 'Periodo',
        'Total': 'Total'
    }

    df.rename(columns=column_rename_dict, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [3]:
def actualizar_valor_en_fila(dataframe, fila, columna, resta_resultado):
    dataframe.loc[fila, columna] = resta_resultado

# AEMET

In [4]:
def obtener_encabezado_api(api_file_path='../api.txt'):
# utilizo un token de autorización guardado en un archivo para acceder a la API de manera segura y obtener datos climáticos actualizados
    aemet = os.getenv("api")

    with open(api_file_path, 'r') as file: 
    
        token = file.read()

    header = {'Authorization': f"password {token}"}
    
    return header

In [5]:
def solicitud_get(url, api_file_path='../api.txt'):
    
    header = obtener_encabezado_api(api_file_path)

    querystring = {"api_key": header['Authorization'].split()[-1]}

    headers = {
        'cache-control': "no-cache"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        return response.json()
    else:
        # Si la solicitud no fue exitosa, imprimir el código de estado y texto de la respuesta
        print(f"Error en la solicitud. Código de estado: {response.status_code}, Mensaje: {response.text}")
        return None

In [6]:
def procesar_datos_mes(url, columnas_interes):
    # Obtener los datos de la API
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta JSON a un DataFrame
        datos = response.json()
        df = pd.DataFrame(datos)
        # Seleccionar columnas de interés
        df = df[columnas_interes]
        
        return df

In [25]:
#limpieza de datos para cada mes

def procesar_mes(df_mes):
    
    # Reemplazar comas por puntos y convertir las columnas numéricas a tipo float
    df_mes['tmed'] = df_mes['tmed'].str.replace(',', '.').astype(float)
    df_mes['prec'] = df_mes['prec'].replace('Acum', np.nan)
    df_mes['prec'] = df_mes['prec'].str.replace(',', '.').replace('Ip', np.nan).astype(float)
    df_mes['sol'] = df_mes['sol'].str.replace(',', '.').astype(float)
    df_mes['tmin'] = df_mes['tmin'].str.replace(',', '.').astype(float)
    df_mes['tmax'] = df_mes['tmax'].str.replace(',', '.').astype(float)
    
    # Convertir la columna "fecha" en datetime
    df_mes['fecha'] = pd.to_datetime(df_mes['fecha'])
    
    #que la columna 'fecha' se vea solo el mes y año
    df_mes['fecha'] = pd.to_datetime(df_mes['fecha'], format='%YM%m').dt.to_period('M')
    
    # Agrupar por provincia y calcular la media de cada columna
    df_media_mes = df_mes.groupby('provincia').agg({
        'fecha':'first',
        'tmed': 'mean',
        'prec': 'mean',
        'sol': 'mean',
        'tmin': 'mean',
        'tmax': 'mean'
    }).reset_index()
    
    # Renombrar las columnas para mayor claridad
    df_media_mes = df_media_mes.rename(columns={
        'fecha':'periodo',
        'tmed': 'media_tmed',
        'prec': 'media_prec',
        'sol': 'media_sol',
        'tmin': 'media_tmin',
        'tmax': 'media_tmax'
    })
    # Eliminar las filas correspondientes a 'CEUTA' y 'MELILLA'
    indices_a_eliminar = df_media_mes[df_media_mes['provincia'].isin(['CEUTA', 'MELILLA'])].index
    df_media_mes = df_media_mes.drop(indices_a_eliminar).reset_index(drop=True)
    
    #Capitalizar el nombre de las columnas
    df_media_mes.columns=df_media_mes.columns.str.capitalize()
    
    #Capitalizar las filas de la columna 'Provincia' 
    df_media_mes['Provincia']=df_media_mes['Provincia'].apply(lambda x: x.capitalize())

    
    return df_media_mes

In [11]:
def procesar_dataframe(dataframe):
    # Capitalizar los nombres de las columnas
    dataframe.columns = [col.capitalize() for col in dataframe.columns]

    # Capitalizar la columna 'Provincia'
    dataframe['Provincia'] = dataframe['Provincia'].apply(lambda x: x.capitalize() if isinstance(x, str) else x)

    # Otra operación que deseas realizar en el DataFrame, por ejemplo, imprimir las primeras filas
    return dataframe

# CONEXIÓN SQL

In [5]:
def crear_cadena_conexion(usuario, ruta_archivo_contraseña, host, puerto):
    with open(ruta_archivo_contraseña, 'r') as file:
        contraseña = file.read().strip()

    cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}'

    return cadena_conexion

In [6]:
def crear_base_de_datos(cadena_conexion, nombre_bd):
    engine = create_engine(cadena_conexion)

    with engine.connect() as connection:
        connection.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bd}")

In [9]:
def cargar_csv_a_mysql(ruta_csv, cadena_conexion, nombre_tabla, nombre_bd):

    try:
        # Cargar el CSV en un DataFrame
        df = pd.read_csv(ruta_csv)
        
        # Crear el motor de conexión
        engine = create_engine(cadena_conexion)
        
        # Seleccionar la base de datos
        with engine.connect() as connection:
            connection.execute(f"USE {nombre_bd}")
        
        # Cargar el DataFrame en la base de datos
        df.to_sql(nombre_tabla, con=engine, if_exists='replace', index=False)

        print(f"Datos cargados en la tabla {nombre_tabla} con éxito.")

    except Exception as e:
        print(f"Error al cargar los datos en MySQL: {e}")